### Load packages and data files from previous notebook

In [1]:
## Imports
from scipy import io
import pandas as pd
import numpy as np
import os
import pandas_profiling

In [2]:
## Load Matlab files
file_path = r'C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\interim'
AMO_file, biomass_file, ecosystem_file, landings_file, ocean_temp_file = [fr'{file_path}\AMO_data.csv', \
    fr'{file_path}\biomass_data.csv', fr'{file_path}\ecosystem_data.csv', fr'{file_path}\landings_data.csv', \
    fr'{file_path}\ocean_temp_data.csv']

AMO_df = pd.read_csv(AMO_file).drop(['Unnamed: 0'], axis=1)
biomass_df = pd.read_csv(biomass_file).drop(['Unnamed: 0'], axis=1)
ecosystem_df = pd.read_csv(ecosystem_file).drop(['Unnamed: 0'], axis=1)
landings_df = pd.read_csv(landings_file).drop(['Unnamed: 0'], axis=1)
ocean_temp_df = pd.read_csv(ocean_temp_file).drop(['Unnamed: 0'], axis=1)

### Generate df reports and save to files

In [19]:
def make_html_report (df, name):
    report = df.profile_report(sort=None, html={'style':{'full_width': True}}, progress_bar=True)
    report.to_file(fr"C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\reports\{name}_data_report.html")
    print(f'{name} done!')

make_html_report(AMO_df, 'AMO')
make_html_report(biomass_df, 'biomass')
make_html_report(ecosystem_df, 'ecosystem')
make_html_report(landings_df, 'landings')
make_html_report(ocean_temp_df, 'ocean_temp')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 29.41it/s]


AMO done!


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 27.02it/s]


biomass done!


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 200.02it/s]


ecosystem done!


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 124.99it/s]


landings done!


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 76.91it/s]

ocean_temp done!


## Data defining:

### In Biomass:
#### biomass in tons per square kilometer (t/km^2).
#### biomass_index in tons.
#### abundance in number per square kilometer.
#### abundance_index in number.

### In Landings:
#### landings_biomass in tons per square kilometer (t/km^2).
#### landings_biomass_index in tons.
#### landings_abundance in number per square kilometer.
#### landings_abundance_index in number.

## Decisions made for data cleaning:

#### Change nothing in ecosystem or ocean temp data. Annualize AMO data instead of month-by-month to AMO_ann_data.csv. All ecosystems in landings_data area also found in biomass_data.

In [40]:
#print(ecosystem_df.head())
#print(AMO_df.head())
#print(ocean_temp_df.head())
print(biomass_df['species'].value_counts())
print(landings_df.dtypes)
#print(biomass_df.loc[(biomass_df['species']=='Sheepshead') & (biomass_df['abundance']>=1)])

Scup                  450
Winter_flounder       447
Summer_flounder       438
Red_hake              425
Silver_hake           418
                     ... 
Houndfish               1
Green_sunfish           1
LADY_FISH               1
LEATHERBACK_TURTLE      1
northern_sennet         1
Name: species, Length: 600, dtype: int64
ecosystem                  object
species                    object
year                        int64
landings_biomass          float64
landings_abund            float64
landings_biomass_index    float64
landings_abund_index      float64
avg_len                   float64
avg_mass                  float64
source                     object
agency                     object
season                     object
dtype: object
